In [20]:
import json
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [2]:
triviaqa_dataset = load_dataset("mandarjoshi/trivia_qa", "rc", split="validation")

Generating test split: 100%|██████████| 17210/17210 [00:02<00:00, 6186.51 examples/s]


In [16]:
kilt_dataset = load_dataset("facebook/kilt_tasks", "triviaqa_support_only", split="validation")

Generating test split: 100%|██████████| 6586/6586 [00:00<00:00, 278254.20 examples/s]


In [17]:
kilt_ids = set(kilt_dataset["id"])

In [18]:
triviaqa_dataset[0]['question_id'], 'qw_9726' in kilt_ids

('tc_2', True)

In [54]:
import json
from tqdm import tqdm

filtered_triviaqa = []
seen_ids = set()

for row in tqdm(triviaqa_dataset):
    q_id = row["question_id"]
    wiki_context = row.get("entity_pages", {}).get("wiki_context")

    if (
        q_id in kilt_ids  # ID must be in kilt_ids
        and q_id not in seen_ids  # Ensure unique IDs
        and isinstance(wiki_context, list)  # Must be a list
        and isinstance(wiki_context[0], str)  # It's first element must be a string
        and wiki_context[0].strip() != ""  # The string must not be empty
    ):
        filtered_triviaqa.append(row)
        seen_ids.add(q_id)

output_file = "kilt_triviaqa_subset.json"
with open(output_file, "w") as f:
    json.dump(filtered_triviaqa, f, indent=4)

print(f"Filtered dataset saved to {output_file}")


100%|██████████| 17944/17944 [00:06<00:00, 2805.08it/s]


Filtered dataset saved to kilt_triviaqa_subset.json


In [55]:
len(filtered_triviaqa), len(kilt_ids)

(5359, 5359)